In [87]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import os
import librosa
import pdb

from os import path
from pydub import AudioSegment

import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import torch
from torch import nn
from torch.utils import data
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.utils.data as utils

from torch.autograd import Variable

import time

In [3]:
'''

# Convert mp3 to wav

path_audio = 'MEMD_audio'
list_mp3 = []

for path, subdirs, files in os.walk(path_audio):
    for filename in files:
        if filename.endswith('.mp3'):
            list_mp3.append(os.path.join(path, filename))

list_mp3 = sorted(list_mp3)
list_mp3.sort(key = lambda f:int(''.join(filter(str.isdigit,f))))

for path in list_mp3:    
    
    src = path
    dst = path[:-4] + '.wav'
                                                          
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="wav")
    
'''

In [92]:
# Load dynamic annotations

path_arousal = 'annotations/annotations averaged per song/dynamic (per second annotations)/arousal.csv'
path_valence = 'annotations/annotations averaged per song/dynamic (per second annotations)/valence.csv'

n_columns = 60
num_files = 1802
specs_per_file = 6

arousal_matrix_files = np.zeros((num_files, n_columns-1))
valence_matrix_files = np.zeros((num_files, n_columns-1))
for n in range(1,n_columns-1):
    arousal_matrix_files[:,n] = np.loadtxt(path_arousal, delimiter=',', usecols=n)
    valence_matrix_files[:,n] = np.loadtxt(path_valence, delimiter=',', usecols=n)

arousal_matrix_frames = np.zeros((num_files, specs_per_file))
valence_matrix_frames = np.zeros((num_files, specs_per_file))

hop_size_ant = 9

for i in range(num_files):
    for j in range(specs_per_file):
        arousal_matrix_frames[i,j] = np.mean(arousal_matrix_files[i,(j*hop_size_ant):((j+1)*hop_size_ant)])
        valence_matrix_frames[i,j] = np.mean(valence_matrix_files[i,(j*hop_size_ant):((j+1)*hop_size_ant)])
        
arousal_vector_frames = arousal_matrix_frames.flatten()
valence_vector_frames = valence_matrix_frames.flatten()

arousal_vector_frames = ((arousal_vector_frames+1)/2)*10
valence_vector_frames = ((valence_vector_frames+1)/2)*10

arousal_vector_frames = arousal_vector_frames.astype(int)
valence_vector_frames = valence_vector_frames.astype(int)

np.save('arousal_vector_frames.npy', arousal_vector_frames)
np.save('valence_vector_frames.npy', valence_vector_frames)

In [67]:
# Load spectrograms

path_audio = 'MEMD_audio'

list_wav = []
for path, subdirs, files in os.walk(path_audio):
    for filename in files:
        if filename.endswith('.wav'):
            list_wav.append(os.path.join(path, filename))

list_wav = sorted(list_wav)
list_wav.sort(key = lambda f:int(''.join(filter(str.isdigit,f))))

start_audio = int(44100*15)

frame_size = 4096
hop_size = 2048
num_freq = 96

audio_matrix_frames = np.zeros((int(num_files*specs_per_file), num_freq, num_freq))

for n in range(num_files):
    
    audio, fs = librosa.load(list_wav[n], sr=44100)
    audio = audio[start_audio:]
    
    spec = librosa.feature.melspectrogram(audio, sr=44100, n_fft=frame_size, hop_length=hop_size, n_mels=num_freq)
    spec = np.log((abs(spec)*1e-2)+1e-16)
    spec = (spec-np.min(spec))/(np.max(spec)-np.min(spec))
    
    for i in range(specs_per_file):
        audio_matrix_frames[(n*specs_per_file)+i] = spec[:,int(i*num_freq):int((i+1)*num_freq)]
        
np.save('audio_matrix_frames.npy', audio_matrix_frames)

In [106]:
# Get deep learning model

def Train_Val_Test_Split(Dataset_Spec, Classes, Percentage_Train, Percentage_Val, Shuffle=True, Embedding=False, Energy=None):

    Percentage_Test = 100. - Percentage_Train - Percentage_Val

    N_Train = int(np.floor(Dataset_Spec.shape[0]*(Percentage_Train/100.)))
    N_Val = int(np.floor(Dataset_Spec.shape[0]*(Percentage_Val/100.)))
    N_Test = int(np.floor(Dataset_Spec.shape[0]*(Percentage_Test/100.)))

    if (N_Train + N_Test + N_Val != Dataset_Spec.shape[0]):
        N_Test += Dataset_Spec.shape[0] - (N_Train + N_Test + N_Val)
        
    Indices = np.arange(Dataset_Spec.shape[0])
    
    if Shuffle:
        np.random.shuffle(Indices)

    Indices_Train = Indices[:N_Train]
    Indices_Val = Indices[N_Train:N_Train+N_Val]
    Indices_Test = Indices[N_Train+N_Val:N_Train+N_Val+N_Test]
    
    Dataset_Spec_Train = Dataset_Spec[Indices_Train]
    Classes_Train = Classes[Indices_Train]
    Dataset_Spec_Val = Dataset_Spec[Indices_Val]
    Classes_Val = Classes[Indices_Val]
    Dataset_Spec_Test = Dataset_Spec[Indices_Test]
    Classes_Test = Classes[Indices_Test]
    
    Tensor_Train = torch.from_numpy(Dataset_Spec_Train).type(torch.FloatTensor)
    Tensor_Val = torch.from_numpy(Dataset_Spec_Val).type(torch.FloatTensor)
    Tensor_Test = torch.from_numpy(Dataset_Spec_Test).type(torch.FloatTensor)
    
    if Embedding:
        Classes_Train = torch.from_numpy(Classes_Train).type(torch.FloatTensor)
        Classes_Val = torch.from_numpy(Classes_Val).type(torch.FloatTensor)
        Classes_Test = torch.from_numpy(Classes_Test).type(torch.FloatTensor)
    else:
        Classes_Train = torch.from_numpy(Classes_Train).type(torch.LongTensor)
        Classes_Val = torch.from_numpy(Classes_Val).type(torch.LongTensor)
        Classes_Test = torch.from_numpy(Classes_Test).type(torch.LongTensor)
        
    if type(Energy)!=type(None):
        Energy_Train = Energy[Indices_Train]
        Energy_Val = Energy[Indices_Val]
        Energy_Test = Energy[Indices_Test]
        Energy_Train = torch.from_numpy(Energy_Train).type(torch.FloatTensor)
        Energy_Val = torch.from_numpy(Energy_Val).type(torch.FloatTensor)
        Energy_Test = torch.from_numpy(Energy_Test).type(torch.FloatTensor)
        return Tensor_Train, Classes_Train, Tensor_Val, Classes_Val, Tensor_Test, Classes_Test, Energy_Train, Energy_Val, Energy_Test, Indices_Test
    else:
        return Tensor_Train, Classes_Train, Tensor_Val, Classes_Val, Tensor_Test, Classes_Test, Indices_Test


class CNN_Fixed(torch.nn.Module):
    
    def __init__(self):
        super(CNN_Fixed, self).__init__()
        
        self.convnet = nn.Sequential(nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1), nn.PReLU(),
                                     nn.Dropout(0.1), nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1), nn.PReLU(),
                                     nn.Dropout(0.1), nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), nn.PReLU(),
                                     nn.Dropout(0.1), nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), nn.PReLU(),
                                     nn.Dropout(0.1), nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), nn.PReLU(),
                                     nn.Dropout(0.1), nn.MaxPool2d(2, stride=2))

        self.fc = nn.Sequential(nn.Linear((128*3*3), 128),
                                nn.PReLU(),
                                nn.Dropout(0.4),
                                nn.Linear(128, 10)
                                )

    def forward(self, x):
        x = x.unsqueeze(1)
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)
    
    
class EarlyStopping:
    
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
    
    def __call__(self, val_loss, model, mode):
        
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, mode)
        elif score < self.best_score - self.delta:
            self.counter += 1
            print('EarlyStopping counter: ' + str(self.counter) + ' out of ' + str(self.patience))
            print('\n')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, mode)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, mode):
        if self.verbose:
            print('Validation loss decreased (' + str(self.val_loss_min) + ' --> ' + str(val_loss) + ').  Saving model ...')
        torch.save(model.state_dict(), 'MinValLossModel_' + mode + '.pth')
        self.val_loss_min = val_loss
        
        
Dataset = np.load('audio_matrix_frames.npy')
Classes = np.load('arousal_vector_frames.npy')

Classes_One_Hot = np.zeros((len(Classes), 10))
for n in range(len(Classes)):
    Classes_One_Hot[n,int(Classes[n])] = 1


###############################################  MODEL 1 - AROUSAL  ###############################################

print('\n')
print('Model 1 - AROUSAL')
print('\n')

start = time.time()

# Set hyperparameters

mode = 'Arousal'

Percentage_Train = 80
Percentage_Val = 10

num_classes = 2
batch_size = 16
learning_rate = 5*1e-4
num_epochs = 100
num_thresholds_F1_score = 10.
patience_early = 3

# Main

Tensor_Train, Labels_Train, Tensor_Val, Labels_Val, Tensor_Test, Labels_Test, Indices_Test = Train_Val_Test_Split(Dataset, Classes_One_Hot, Percentage_Train, Percentage_Val, Shuffle=False, Embedding=False, Energy=None)
np.save('Indices_Test.npy', Indices_Test)

model_cnn_class = CNN_Fixed()

Train_Dataset = utils.TensorDataset(Tensor_Train, Labels_Train)
Val_Dataset = utils.TensorDataset(Tensor_Val, Labels_Val)
Test_Dataset = utils.TensorDataset(Tensor_Test, Labels_Test)

Train_Loader = torch.utils.data.DataLoader(Train_Dataset, batch_size=batch_size, shuffle=True)
Val_Loader = torch.utils.data.DataLoader(Val_Dataset, batch_size=Tensor_Val.size()[0], shuffle=False)
Test_Loader = torch.utils.data.DataLoader(Test_Dataset, batch_size=Tensor_Test.size()[0], shuffle=False)

optimizer = torch.optim.Adam(model_cnn_class.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
early_stopping = EarlyStopping(patience=patience_early, verbose=False)

loss = torch.nn.BCEWithLogitsLoss()
sig = torch.nn.Sigmoid()
sfmx = torch.nn.LogSoftmax()

for epoch in range(num_epochs):

    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 11)

    train_loss = 0.
    validation_loss = 0.

    validation_accuracy = 0.

    count_batch_train = 0.

    for batch in Train_Loader:

        data, classes = batch

        classes = classes.double()

        predictions = model_cnn_class(data)

        predictions = predictions.squeeze(-1)
        predictions = predictions.double()
        t_loss = loss(predictions, classes)

        optimizer.zero_grad()
        t_loss.backward()
        optimizer.step()

        train_loss += t_loss.item()

        count_batch_train += 1.

    count_batch_val = 0.

    for batch in Val_Loader:

        with torch.no_grad():

            data, classes = batch

            classes = classes.double()

            predictions = model_cnn_class(data)

            predictions = predictions.squeeze(-1)
            predictions = predictions.double()
            v_loss = loss(predictions, classes)

            optimizer.zero_grad()

            validation_loss += v_loss.item()

            scheduler.step(validation_loss)

            count_batch_val += 1.

    train_loss /= float(count_batch_train)
    validation_loss /= float(count_batch_val)

    print('Train Loss: {:.4f}, Validation Loss: {:.4f}'.format(train_loss, validation_loss))
    print('\n')

    early_stopping(validation_loss, model_cnn_class, mode)

    if early_stopping.early_stop:
        print("Early stopping")
        break

torch.save(model_cnn_class.state_dict(),  'LastModel_' + mode + '.pth')



Model 1 - AROUSAL


Epoch 1/100
-----------
Train Loss: 0.2700, Validation Loss: 0.2238


Epoch 2/100
-----------
Train Loss: 0.2377, Validation Loss: 0.2272


EarlyStopping counter: 1 out of 3


Epoch 3/100
-----------
Train Loss: 0.2317, Validation Loss: 0.2326


EarlyStopping counter: 2 out of 3


Epoch 4/100
-----------
Train Loss: 0.2273, Validation Loss: 0.2068


Epoch 5/100
-----------
Train Loss: 0.2229, Validation Loss: 0.2202


EarlyStopping counter: 1 out of 3


Epoch 6/100
-----------
Train Loss: 0.2197, Validation Loss: 0.2163


EarlyStopping counter: 2 out of 3


Epoch 7/100
-----------
Train Loss: 0.2161, Validation Loss: 0.2140


EarlyStopping counter: 3 out of 3


Early stopping


In [119]:
# Testing

model_cnn_class.load_state_dict(torch.load('MinValLossModel_' + mode + '.pth'))

test_accuracy = 0.
count_batch_test = 0.

with torch.no_grad():

    for batch in Test_Loader:

        data, classes = batch

        classes = classes.double()

        model_cnn_class.eval()

        predictions = model_cnn_class(data)

        predictions = predictions.squeeze(-1)
        predictions = predictions.double()

        optimizer.zero_grad()

        count_batch_test += 1.

sfmx = torch.nn.LogSoftmax()
for idx in range(len(data)):
    print(classes[idx].argmax())
    print(sfmx(predictions[idx]).argmax())

tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(8)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(6)
tensor(7)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(5)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(7)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(4)
tensor(4)
tensor(4)
tensor(5)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)


/home/alejandro/anaconda3/envs/mip/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(5)
tensor(4)
tensor(5)
tensor(4)
tensor(5)
tensor(5)
tensor(5)
tensor(6)
tensor(5)
tensor(4)
tensor(4)
tensor(4)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(6)
tensor(6)


In [127]:
Dataset = np.load('audio_matrix_frames.npy')
Classes = np.load('valence_vector_frames.npy')

Classes_One_Hot = np.zeros((len(Classes), 10))
for n in range(len(Classes)):
    Classes_One_Hot[n,int(Classes[n])] = 1


###############################################  MODEL 1 - VALENCE  ###############################################

print('\n')
print('Model 1 - VALENCE')
print('\n')

start = time.time()

# Set hyperparameters

mode = 'Valence'

Percentage_Train = 80
Percentage_Val = 10

num_classes = 2
batch_size = 16
learning_rate = 1e-3
num_epochs = 100
num_thresholds_F1_score = 10.
patience_early = 3

# Main

Tensor_Train, Labels_Train, Tensor_Val, Labels_Val, Tensor_Test, Labels_Test, Indices_Test = Train_Val_Test_Split(Dataset, Classes_One_Hot, Percentage_Train, Percentage_Val, Shuffle=False, Embedding=False, Energy=None)
np.save('Indices_Test.npy', Indices_Test)

model_cnn_class = CNN_Fixed()

Train_Dataset = utils.TensorDataset(Tensor_Train, Labels_Train)
Val_Dataset = utils.TensorDataset(Tensor_Val, Labels_Val)
Test_Dataset = utils.TensorDataset(Tensor_Test, Labels_Test)

Train_Loader = torch.utils.data.DataLoader(Train_Dataset, batch_size=batch_size, shuffle=True)
Val_Loader = torch.utils.data.DataLoader(Val_Dataset, batch_size=Tensor_Val.size()[0], shuffle=False)
Test_Loader = torch.utils.data.DataLoader(Test_Dataset, batch_size=Tensor_Test.size()[0], shuffle=False)

optimizer = torch.optim.Adam(model_cnn_class.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
early_stopping = EarlyStopping(patience=patience_early, verbose=False)

loss = torch.nn.BCEWithLogitsLoss()
sig = torch.nn.Sigmoid()
sfmx = torch.nn.LogSoftmax()

for epoch in range(num_epochs):

    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 11)

    train_loss = 0.
    validation_loss = 0.

    validation_accuracy = 0.

    count_batch_train = 0.

    for batch in Train_Loader:

        data, classes = batch

        classes = classes.double()

        predictions = model_cnn_class(data)

        predictions = predictions.squeeze(-1)
        predictions = predictions.double()
        t_loss = loss(predictions, classes)

        optimizer.zero_grad()
        t_loss.backward()
        optimizer.step()

        train_loss += t_loss.item()

        count_batch_train += 1.

    count_batch_val = 0.

    for batch in Val_Loader:

        with torch.no_grad():

            data, classes = batch

            classes = classes.double()

            predictions = model_cnn_class(data)

            predictions = predictions.squeeze(-1)
            predictions = predictions.double()
            v_loss = loss(predictions, classes)

            optimizer.zero_grad()

            validation_loss += v_loss.item()

            scheduler.step(validation_loss)

            count_batch_val += 1.

    train_loss /= float(count_batch_train)
    validation_loss /= float(count_batch_val)

    print('Train Loss: {:.4f}, Validation Loss: {:.4f}'.format(train_loss, validation_loss))
    print('\n')

    early_stopping(validation_loss, model_cnn_class, mode)

    if early_stopping.early_stop:
        print("Early stopping")
        break

torch.save(model_cnn_class.state_dict(),  'LastModel_' + mode + '.pth')



Model 1 - VALENCE


Epoch 1/100
-----------
Train Loss: 0.2557, Validation Loss: 0.2298


Epoch 2/100
-----------
Train Loss: 0.2356, Validation Loss: 0.2305


EarlyStopping counter: 1 out of 3


Epoch 3/100
-----------
Train Loss: 0.2312, Validation Loss: 0.2264


Epoch 4/100
-----------
Train Loss: 0.2279, Validation Loss: 0.2286


EarlyStopping counter: 1 out of 3


Epoch 5/100
-----------
Train Loss: 0.2258, Validation Loss: 0.2415


EarlyStopping counter: 2 out of 3


Epoch 6/100
-----------
Train Loss: 0.2224, Validation Loss: 0.2361


EarlyStopping counter: 3 out of 3


Early stopping


In [128]:
# Testing

model_cnn_class.load_state_dict(torch.load('MinValLossModel_' + mode + '.pth'))

test_accuracy = 0.
count_batch_test = 0.

with torch.no_grad():

    for batch in Test_Loader:

        data, classes = batch

        classes = classes.double()

        model_cnn_class.eval()

        predictions = model_cnn_class(data)

        predictions = predictions.squeeze(-1)
        predictions = predictions.double()

        optimizer.zero_grad()

        count_batch_test += 1.

sfmx = torch.nn.LogSoftmax()
for idx in range(len(data)):
    print(classes[idx].argmax())
    print(sfmx(predictions[idx]).argmax())

tensor(7)
tensor(6)
tensor(7)
tensor(5)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(5)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(6)
tensor(6)
tensor(5)
tensor(6)
tensor(6)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(5)
tensor(7)
tensor(5)
tensor(7)
tensor(5)
tensor(7)
tensor(5)
tensor(7)
tensor(5)
tensor(3)
tensor(5)
tensor(3)
tensor(5)
tensor(3)
tensor(5)
tensor(3)
tensor(5)
tensor(3)
tensor(5)
tensor(3)
tensor(5)
tensor(4)
tensor(4)
tensor(4)
tensor(5)
tensor(3)
tensor(4)
tensor(4)
tensor(4)
tensor(3)
tensor(5)
tensor(3)
tensor(4)


/home/alejandro/anaconda3/envs/mip/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(4)
tensor(6)
tensor(2)
tensor(5)
tensor(2)
tensor(4)
tensor(2)
tensor(4)
tensor(2)
tensor(4)
tensor(2)
tensor(4)
tensor(2)
tensor(4)
tensor(3)
tensor(4)
tensor(3)
tensor(5)
tensor(3)
tensor(4)
tensor(3)
tensor(5)
tensor(3)
tensor(4)
tensor(2)
tensor(5)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(3)
tensor(6)
tensor(3)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(4)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(6)
tensor(7)
tensor(5)
tensor(7)
tensor(6)
tensor(7)
tensor(6)
tensor(7)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(6)
tensor(5)
tensor(6)
tensor(5)
tensor(5)
tensor(5)
tensor(6)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)
tensor(5)


In [129]:
np.save('Tensor_Test.npy', Tensor_Test.numpy())
np.save('Labels_Test.npy', Labels_Test.numpy())